# CCMP Winds in a cloud-optimized-format for Pangeo

The Cross-Calibrated Multi-Platform (CCMP) Ocean Surface Wind Vector Analyses is part of the NASA Making Earth System Data Records for Use in Research Environments (MEaSUREs) Program. MEaSUREs, develops consistent global- and continental-scale Earth System Data Records by supporting projects that produce data using proven algorithms and input.  If you use this data, please give [credit](https://podaac.jpl.nasa.gov/MEaSUREs-CCMP?sections=about).  For more information, please review the [documentation](https://podaac-tools.jpl.nasa.gov/drive/files/allData/ccmp/L2.5/docs/ccmp_users_guide.pdf).

# Accessing cloud satellite data

- CCMP zarr conversion funding: Interagency Implementation and Advanced Concepts Team [IMPACT](https://earthdata.nasa.gov/esds/impact) for the Earth Science Data Systems (ESDS) program and AWS Public Dataset Program
  
### Credits: Tutorial development
* [Dr. Chelle Gentemann](mailto:gentemann@faralloninstitute.org) -  [Twitter](https://twitter.com/ChelleGentemann)   - Farallon Institute

### Zarr data format

 [Zarr](https://zarr.readthedocs.io/en/stable/)

### Data proximate computing
These are BIG datasets that you can analyze on the cloud without downloading the data. You can run this on your phone, a Raspberry Pi, laptop, or desktop.   
By using public cloud data, your science is reproducible and easily shared!

### To run this notebook

Code is in the cells that have <span style="color: blue;">In [  ]:</span> to the left of the cell and have a colored background

To run the code:
- option 1) click anywhere in the cell, then hold `shift` down and press `Enter`
- option 2) click on the Run button at the top of the page in the dashboard

Remember:
- to insert a new cell below press `Esc` then `b`
- to delete a cell press `Esc` then `dd`

### First start by importing libraries

In [ ]:
#libs for reading data
import xarray as xr
import gcsfs
import glob
import numpy as np
import matplotlib.pyplot as plt

#lib for dask gateway
from dask_gateway import Gateway
from dask.distributed import Client

### Start a cluster, a group of computers that will work together.

(A cluster is the key to big data analysis on on Cloud.)

- This will set up a [dask kubernetes](https://docs.dask.org/en/latest/setup/kubernetes.html) cluster for your analysis and give you a path that you can paste into the top of the Dask dashboard to visualize parts of your cluster.  
- You don't need to paste the link below into the Dask dashboard for this to work, but it will help you visualize progress.
- Try 20 workers to start (during the tutorial) but you can increase to speed things up later

In [ ]:
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=20)
client = Client(cluster)
cluster

** ☝️ Don’t forget to click the link above or copy it to the Dask dashboard ![images.png](attachment:images.png) on the left to view the scheduler dashboard! **

### Initialize Dataset

Here we load the dataset from the zarr store. Note that this very large dataset (273 GB) initializes nearly instantly, and we can see the full list of variables and coordinates.

### Examine Metadata

For those unfamiliar with this dataset, the variable metadata is very helpful for understanding what the variables actually represent
Printing the dataset will show you the dimensions, coordinates, and data variables with clickable icons at the end that show more metadata and size.

In [ ]:
zstore = 'gs://pangeo-nasa-ccmp/zarr'

fs = gcsfs.GCSFileSystem(project='pangeo-181919',requester_pays=True)

ds = xr.open_zarr(fs.get_mapper(zstore))

ds['wspd']=np.sqrt(ds.uwnd**2+ds.vwnd**2)

ds

# Plot a global image of the data on 7/28/2020

``xarray`` makes plotting the data very easy.  A nice overview of plotting with xarray is [here](http://xarray.pydata.org/en/stable/plotting.html).  Details on [.plot](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.plot.html#xarray.DataArray.plot)

In [ ]:
day = ds.sel(time='2020-07-04T00')

day.wspd.plot()

## Make a land/ocean/ice mask to show where there is actually data

- sum over time for nobs (number of observations) variable

In [ ]:
total_nobs = ds.nobs.sum({'time'},keep_attrs=True)

total_nobs


# Apply the mask 
- only show data (over the ocean) where there are satellite observations

In [ ]:
masked = day.where(total_nobs>0)

masked.wspd.plot()

# Plot a timeseries at a specific point

In [ ]:
ds.wspd.sel(latitude=45,longitude=210,method='nearest').plot()

### Using ``.groupby`` and ``.resample``
#### Create a monthly anomaly dataset

- First create a climatology ``.groupby``.  Two nice arguments for ``.mean``: ``keep_addrs`` which keeps the metadata and ``skipna`` which ensures that only data that is always present is included.
- Calculate the anomaly (difference from a climatology) using ``.groupby``
- Calculate a monthly average anomaly using ``.resample``


In [ ]:
%%time
ds_climatology = ds.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)

ds_anomaly = ds.groupby('time.dayofyear')-ds_climatology

ds_anomaly_monthly = ds_anomaly.resample(time='1MS').mean()

# make a global mean timeseries of the anomaly

In [ ]:
%%time
ts = ds_anomaly.mean({'latitude','longitude'})

ts_mon = ds_anomaly_monthly.mean({'latitude','longitude'})

ts

In [ ]:
%%time

ts_mon.wspd.plot()